In [1]:
%pip install langchain==0.3.23 | tail -n 1 
%pip install langchain-ibm==0.3.10 | tail -n 1 
%pip install langchain-community==0.3.16 | tail -n 1 
%pip install wikipedia==1.4.0 | tail -n 1
%pip install openai==1.77.0 | tail -n 1
%pip install langchain-openai==0.3.16 | tail -n 1
%pip install langgraph==0.6.1 | tail -n 1

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_ibm import ChatWatsonx
from langchain.agents import AgentType


In [2]:
llm = ChatWatsonx(
    model_id="ibm/granite-3-2-8b-instruct",
    url="https://us-south.ml.cloud.ibm.com",
    project_id="skills-network",
)

In [3]:
response = llm.invoke("What framework do you recommend for building AI Agents - LangChain or LangGraph?")
print("\nResponse Content: ", response.content)


Response Content:  Both LangChain and LangGraph are interesting concepts in AI but they serve quite different purposes and are at different stages of development.

1. **LangChain**: This is a relatively new concept, introduced by researchers at Microsoft. It's a framework for building generalist, few-shot learning agents. It's designed to understand and execute instructions in a natural language, demonstrating a mix of multiple skills. It's still in the research phase and not yet widely available or production-ready. 

2. **LangGraph**: This isn't a common AI framework that I'm aware of. It could potentially be a typo or a lesser-known system. If it's mistyped, it might be referring to concepts like "Graph Neural Networks" which are a type of neural network designed to perform well on graph-structured data. If it's a specific AI system or framework, more information would be needed for a precise response.

Currently, if you're looking to build AI agents, more established and well-docu

In [4]:
from langgraph.prebuilt import create_react_agent

In [5]:
from langchain_core.tools import tool
import re

In [6]:
@tool
def sum_numbers_from_text(inputs: str) -> float:
    """
    Adds a list of numbers provided in the input string.
    
    Args:
        text: A string containing numbers that should be extracted and summed.
        
    Returns:
        The sum of all numbers found in the input.
    """
    # Use regular expressions to extract all numbers from the input
    numbers = [int(num) for num in re.findall(r'\d+', inputs)]
    result = sum(numbers)
    return result

In [7]:
print("Name: \n", sum_numbers_from_text.name)
print("Description: \n", sum_numbers_from_text.description) 
print("Args: \n", sum_numbers_from_text.args) 

Name: 
 sum_numbers_from_text
Description: 
 Adds a list of numbers provided in the input string.

Args:
    text: A string containing numbers that should be extracted and summed.

Returns:
    The sum of all numbers found in the input.
Args: 
 {'inputs': {'title': 'Inputs', 'type': 'string'}}


In [8]:
agent_exec = create_react_agent(model=llm, tools=[sum_numbers_from_text])
msgs = agent_exec.invoke({"messages": [("human", "Add the numbers 10, 20, 30")]})

In [9]:
print(msgs["messages"][-1].content)

The sum of the numbers 10, 20, and 30 is 60.


In [10]:
from langchain_community.utilities import WikipediaAPIWrapper

# Create a Wikipedia tool using the @tool decorator
@tool
def search_wikipedia(query: str) -> str:
    """Search Wikipedia for factual information about a topic.
    
    Parameters:
    - query (str): The topic or question to search for on Wikipedia
    
    Returns:
    - str: A summary of relevant information from Wikipedia
    """
    wiki = WikipediaAPIWrapper()
    return wiki.run(query)

In [11]:
search_wikipedia.invoke("What is the population of Papua New Guinea?")

'Page: Demographics of Papua New Guinea\nSummary: The indigenous population of Papua New Guinea is one of the most heterogeneous in the world. Papua New Guinea has several thousand separate communities, most with only a few hundred people. Divided by language, customs, and tradition, some of these communities have engaged in endemic warfare with their neighbors for centuries. It is the second most populous nation in Oceania, with a total population estimated variously as being between 9.5 and 10.1 million inhabitants.\n\nThe isolation created by the mountainous terrain is so great that some groups, until recently, were unaware of the existence of neighboring groups only a few kilometers away. The diversity, reflected in a folk saying, "For each village, a different culture", is perhaps best shown in the local languages. The island of New Guinea contains about 850 languages. The languages that are neither Austronesian nor Australian are considered Papuan languages; this is a geographica

In [12]:
# Update your tools list to include the Wikipedia tool
tools_updated = [sum_numbers_from_text, search_wikipedia]

# Create the agent with all tools including Wikipedia
math_agent_updated = create_react_agent(
    model=llm,
    tools=tools_updated,
    prompt="You are a helpful assistant that can perform various mathematical operations and look up information. Use the tools precisely and explain your reasoning clearly."
)

In [13]:
query = "What is the population of Papua New Guinea? Add another million to it"

response = math_agent_updated.invoke({"messages": [("human", query)]})

print("\nMessage sequence:")
for i, msg in enumerate(response["messages"]):
    print(f"\n--- Message {i+1} ---")
    print(f"Type: {type(msg).__name__}")
    if hasattr(msg, 'content'):
        print(f"Content: {msg.content}")
    if hasattr(msg, 'name'):
        print(f"Name: {msg.name}")
    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        print(f"Tool calls: {msg.tool_calls}")


Message sequence:

--- Message 1 ---
Type: HumanMessage
Content: What is the population of Papua New Guinea? Add another million to it
Name: None

--- Message 2 ---
Type: AIMessage
Content: 
Name: None
Tool calls: [{'name': 'search_wikipedia', 'args': {'query': 'Population of Papua New Guinea'}, 'id': 'chatcmpl-tool-6a2724906a4d479cbf6c6499cc86f944', 'type': 'tool_call'}]

--- Message 3 ---
Type: ToolMessage
Content: Page: Demographics of Papua New Guinea
Summary: The indigenous population of Papua New Guinea is one of the most heterogeneous in the world. Papua New Guinea has several thousand separate communities, most with only a few hundred people. Divided by language, customs, and tradition, some of these communities have engaged in endemic warfare with their neighbors for centuries. It is the second most populous nation in Oceania, with a total population estimated variously as being between 9.5 and 10.1 million inhabitants.

The isolation created by the mountainous terrain is so 